<a href="https://colab.research.google.com/github/Shreeshambav/DeepLearning_training/blob/main/Deeplearning_Budget_and_Forecasting_LSTMv1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Budget_merged_file_Cat_with_outliers_MRL_LINE_replaced.xlsx to Budget_merged_file_Cat_with_outliers_MRL_LINE_replaced.xlsx


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix,accuracy_score
import tensorflow as tf
tf.config.run_functions_eagerly(True)
# Read the data
# file_path = 'E:\\Raj\\Book-working\\10-Python\\Program\\Alpha_Optimum_Python\\Projects\\DS\\DL\\Sunoida\\Working_data\\Budget_merged_file_Cat_with_outliers_MRL_LINE_replaced.xlsx'
# df = pd.read_excel(file_path)

file_path = 'Budget_merged_file_Cat_with_outliers_MRL_LINE_replaced.xlsx'
df = pd.read_excel(file_path)

# Drop rows with missing values in the specified columns
df = df.dropna(subset=["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"])

# Convert the values in columns "01" to "12" to numeric
df[["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]] = df[
    ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
].apply(pd.to_numeric, errors="coerce")

# Define months and years
months = [f"{i:02d}" for i in range(1, 13)]
years = df["Year"].unique()

# Group the data based on categories
grouped_df = df.groupby("MRL_LINE")

# Filter data for each year (2019, 2020, 2021) and apply ARIMA model for each MRL_LINE
years = [2019, 2020, 2021, 2022]

# Helper function to find the last financial year
def find_last_financial_year(years):
    sorted_years = sorted(years, reverse=True)
    for i in range(len(sorted_years) - 1):
        if sorted_years[i] - 1 != sorted_years[i + 1]:
            return sorted_years[i]
    return sorted_years[-1]

# Initialize dataframe to store forecast results
predicted_df = pd.DataFrame(columns=["MRL_LINE", "Year", "Month", "LSTM_Predicted_Value"])
forecast_df = pd.DataFrame(columns=["MRL_LINE", "Year", "Month", "LSTM_Forecasted_Value"])
metrics_df = pd.DataFrame(
    columns=["MRL", "MSE", "RMSE", "MAPE", "z_scores_t", "z_scores_p", "R2"]
)

# Initialize lists to store evaluation metrics
mse_list = []
r2_list = []
rmse_list = []
mape_list = []
z_scores_list_t = []
z_scores_list_p = []

def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps + 1):
        X.append(data[i:i + n_steps])
        y.append(data[i + n_steps - 1])  # Change this line
    return np.array(X), np.array(y)


#############################################################
n_steps_train = 24  # Define the number of time steps for training sequences
n_steps_test = 12   # Define the number of time steps for testing sequences

# for mrl_line, group_df in grouped_df:
#     print(f"Processing MRL_LINE: {mrl_line}")
#     print(group_df)  # Print the group DataFrame for debugging

#     for year in years:
#         print(f"Processing Year: {year}")
#         year_df = group_df[group_df['Year'] == year]

#         # Check if all 12 columns exist in the DataFrame
#         data_columns = [str(month).zfill(2) for month in range(1, 13)]
#         missing_columns = [col for col in data_columns if col not in year_df.columns]

#         if missing_columns:
#             print(f"Missing data columns for MRL_LINE: {mrl_line}, Year: {year}: {missing_columns}")
#             continue

#         data = year_df[data_columns].values.flatten()

#         # Handle cases where there might be missing data for some years
#         if np.any((data != 0) & pd.notnull(data)):
#             # Find last financial year
#             last_financial_year = find_last_financial_year(years)

#             # Split data into train and test sets
#             if year == last_financial_year:
#                 # Last 12 months as test data
#                 train_data = data[:-12]
#                 test_data = data[-12:]
#             else:
#                 # Rest as train data
#                 train_data = data[:-12]  # Use all available data except the last 12 months
#                 test_data = data[-12:]  # Use the last 12 months as test data
#                 print("Printing X_train and X_test...")
#                 print(train_data)
#                 print(test_data)

#             # Check if there is enough data to create sequences
#             if len(train_data) >= n_steps_train and len(test_data) >= n_steps_test:

#                 print("Creating sequences...")
#                 print(len(train_data), len(test_data))
#                 # Normalize the data
#                 scaler = MinMaxScaler()
#                 train_data = scaler.fit_transform(train_data.reshape(-1, 1))
#                 test_data = scaler.transform(test_data.reshape(-1, 1))

#                 # Create sequences of data
#                 X_train, y_train = create_sequences(train_data, n_steps_train)
#                 X_test, y_test = create_sequences(test_data, n_steps_test)

#                 # Reshape the input data
#                 X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
#                 X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#                 print("Printing X_train and X_test... reshape")
#                 print(X_train.shape)
#                 print(X_test.shape)



# #################################################
for mrl_line, group_df in grouped_df:
    print(f"Processing MRL_LINE: {mrl_line}")
    print(group_df)  # Print the group DataFrame for debugging

    for year in years:
        print(f"Processing Year: {year}")
        year_df = group_df[group_df['Year'] == year]

        # Check if all 12 columns exist in the DataFrame
        data_columns = [str(month).zfill(2) for month in range(1, 13)]
        missing_columns = [col for col in data_columns if col not in year_df.columns]

        if missing_columns:
            print(f"Missing data columns for MRL_LINE: {mrl_line}, Year: {year}: {missing_columns}")
            continue

        data = year_df[data_columns].values.flatten()

        # Handle cases where there might be missing data for some years
        if np.any((data != 0) & pd.notnull(data)):
            # Find last financial year
            last_financial_year = find_last_financial_year(years)

            # Split data into train and test sets
            if year == last_financial_year:
                # Last 12 months as test data
                train_data = data[:-12]
                test_data = data[-12:]
            else:
                # Rest as train data
                train_data = data[:-12]  # Use all available data except the last 12 months
                test_data = data[-12:]  # Use the last 12 months as test data
                print("Printing X_train and X_test...")
                print(train_data)
                print(test_data)

            # Check if there is enough data to create sequences
            if len(train_data) >= n_steps_train and len(test_data) >= n_steps_test:

                print("Creating sequences...")
                print(len(train_data), len(test_data))
                # Normalize the data
                scaler = MinMaxScaler()
                train_data = scaler.fit_transform(train_data.reshape(-1, 1)).flatten()  # Reshape to 1D
                test_data = scaler.transform(test_data.reshape(-1, 1)).flatten()  # Reshape to 1D

                # Create sequences of data
                X_train, y_train = create_sequences(train_data, n_steps_train)
                X_test, y_test = create_sequences(test_data, n_steps_test)

                # Reshape the input data
                X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
                X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
                print("Printing X_train and X_test... reshape")
                print(X_train.shape)
                print(X_test.shape)
                # # Build the Bidirectional LSTM model
                model = Sequential()
                model.add(
                    Bidirectional(LSTM(units=128, activation="Softmax"), input_shape=(X_train.shape[1], X_train.shape[2]))
                )
                model.add(Dense(1))  # Output layer

                # Compile the model
                model.compile(optimizer="adam", loss="mean_squared_error", metrics=['accuracy'])
                print(model.summary())
                # model.compile(optimizer="adam", loss="mean_squared_error", run_eagerly=True)
                # Early stopping to prevent overfitting
                early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
                # Train the model
                model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)


Processing MRL_LINE: M621100
      COUNTRY  VISION_OUC CURRENCY MRL_LINE          01          02  \
286        KE  KE01008014      KES  M621100        0.00        0.00   
3184       KE  KE01008010      KES  M621100        0.00        0.00   
3854       KE  KE01008012      KES  M621100        0.00        0.00   
5151       KE  KE01008120      KES  M621100        0.00        0.00   
5419       KE  KE01000999      KES  M621100        0.00        0.00   
6913       KE  KE01008053      KES  M621100        0.00        0.00   
8618       KE  KE01008120      KES  M621100        0.00        0.00   
10218      KE  KE01008053      KES  M621100        0.00      201.60   
13397      KE  KE01008010      KES  M621100        0.00  6071325.00   
13476      KE  KE01008120      KES  M621100 -6071325.00  -508848.00   
14595      KE  KE01008120      KES  M621100        0.00        0.00   
15682      KE  KE01000999      KES  M621100        0.00        0.00   
18098      KE  KE01008012      KES  M621100 -936

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


None
Epoch 1/100
2/2 [==============================] - 1s 496ms/step - loss: 0.7718 - accuracy: 0.0256 - val_loss: 0.9597 - val_accuracy: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 1s 617ms/step - loss: 0.7650 - accuracy: 0.0256 - val_loss: 0.9516 - val_accuracy: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 1s 723ms/step - loss: 0.7580 - accuracy: 0.0256 - val_loss: 0.9436 - val_accuracy: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 1s 731ms/step - loss: 0.7512 - accuracy: 0.0256 - val_loss: 0.9355 - val_accuracy: 0.0000e+00
Epoch 5/100
2/2 [==============================] - 1s 462ms/step - loss: 0.7444 - accuracy: 0.0256 - val_loss: 0.9276 - val_accuracy: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 1s 463ms/step - loss: 0.7376 - accuracy: 0.0256 - val_loss: 0.9197 - val_accuracy: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 1s 453ms/step - loss: 0.7309 - accuracy: 0.0256 - val_loss: 0.9118 - val_a

Processing Year: 2020
Printing X_train and X_test...
[ 0.00000000e+00  2.01600000e+02  6.88280000e+02 -7.10635700e+02
  1.02428000e+03  8.66470000e+02  9.72350000e+02  1.22588000e+03
  1.44988000e+03  1.24828000e+03  1.57715000e+03  1.82254000e+03
  0.00000000e+00  6.07132500e+06  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -6.07132500e+06 -5.08848000e+05  0.00000000e+00 -6.78624915e+06
  0.00000000e+00  0.00000000e+00 -6.85851300e+06  0.00000000e+00
 -2.83000000e+04 -9.78167600e+06  0.00000000e+00 -8.11707900e+06
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3.63874500e+06  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -9.8

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


...

None
Epoch 1/100
15/15 [==============================] - 5s 330ms/step - loss: 0.8663 - accuracy: 0.0022 - val_loss: 0.8547 - val_accuracy: 0.0000e+00
Epoch 2/100
15/15 [==============================] - 5s 361ms/step - loss: 0.8096 - accuracy: 0.0022 - val_loss: 0.7982 - val_accuracy: 0.0000e+00
Epoch 3/100
15/15 [==============================] - 5s 343ms/step - loss: 0.7552 - accuracy: 0.0022 - val_loss: 0.7441 - val_accuracy: 0.0000e+00
Epoch 4/100
15/15 [==============================] - 5s 348ms/step - loss: 0.7033 - accuracy: 0.0022 - val_loss: 0.6922 - val_accuracy: 0.0000e+00
Epoch 5/100
15/15 [==============================] - 5s 319ms/step - loss: 0.6535 - accuracy: 0.0022 - val_loss: 0.6426 - val_accuracy: 0.0000e+00
Epoch 6/100
15/15 [==============================] - 5s 321ms/step - loss: 0.6058 - accuracy: 0.0022 - val_loss: 0.5950 - val_accuracy: 0.0000e+00
Epoch 7/100
15/15 [==============================] - 5s 347ms/step - loss: 0.5602 - accuracy: 0.0022 - val_loss: 

Processing Year: 2022
Processing MRL_LINE: M625080
      COUNTRY  VISION_OUC CURRENCY MRL_LINE   01   02   03      04      05  \
89         KE  KE01000014      KES  M625080  0.0  0.0  0.0    0.00    0.00   
384        KE  KE01008100      KES  M625080  0.0  0.0  0.0    0.00    0.00   
989        KE  KE01000011      KES  M625080  0.0  0.0  0.0    0.00    0.00   
1344       KE  KE01000024      KES  M625080  0.0  0.0  0.0    0.00    0.00   
1564       KE  KE01008075      KES  M625080  0.0  0.0  0.0    0.00    0.00   
...       ...         ...      ...      ...  ...  ...  ...     ...     ...   
19041      KE  KE01000030      KES  M625080  0.0  0.0  0.0    0.00    0.00   
19042      KE  KE01008013      KES  M625080  0.0  0.0  0.0    0.00    0.00   
19043      KE  KE01008131      KES  M625080  0.0  0.0  0.0   90.00   90.00   
19044      KE  KE01008134      KES  M625080  0.0  0.0  0.0  216.97  122.78   
19045      KE  KE01008042      KES  M625080  0.0  0.0  0.0    0.00    0.00   

            

Model: "sequential_407"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_406 (Bidirect  (None, 256)              133120    
 ional)                                                          
                                                                 
 dense_406 (Dense)           (None, 1)                 257       
                                                                 
Total params: 133,377
Trainable params: 133,377
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


None
Epoch 1/100
11/11 [==============================] - 4s 349ms/step - loss: 0.9654 - accuracy: 0.0031 - val_loss: 0.9346 - val_accuracy: 0.0000e+00
Epoch 2/100
11/11 [==============================] - 4s 321ms/step - loss: 0.9209 - accuracy: 0.0031 - val_loss: 0.8910 - val_accuracy: 0.0000e+00
Epoch 3/100
11/11 [==============================] - 3s 321ms/step - loss: 0.8776 - accuracy: 0.0031 - val_loss: 0.8487 - val_accuracy: 0.0000e+00
Epoch 4/100
11/11 [==============================] - 4s 341ms/step - loss: 0.8358 - accuracy: 0.0031 - val_loss: 0.8076 - val_accuracy: 0.0000e+00
Epoch 5/100
11/11 [==============================] - 4s 356ms/step - loss: 0.7952 - accuracy: 0.0031 - val_loss: 0.7678 - val_accuracy: 0.0000e+00
Epoch 6/100
11/11 [==============================] - 4s 340ms/step - loss: 0.7557 - accuracy: 0.0031 - val_loss: 0.7292 - val_accuracy: 0.0000e+00
Epoch 7/100
11/11 [==============================] - 4s 336ms/step - loss: 0.7175 - accuracy: 0.0031 - val_loss: 